In [9]:
import os
from os.path import expanduser
import socket
import sys
import glob

# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemOLLIE = 'ollie0'
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "pyfesom")
elif running_system[0:1] == "o" or running_system[0:1] == "b" or running_system == systemHLRN or running_system == systemOLLIE:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"pyfesom")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules") # add custom Vibe 's modules
#sys.path.append("pyfesom") # add standard 's modules
#########################################

computer =  ollie0
working directory set to /home/ollie/loziel/pyfesom


In [10]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
import pyfesom as pf
#from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import cmocean as cmaps
from matplotlib.colors import ListedColormap
import colormaps as cmaps

In [11]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = ListedColormap(cmaps.viridis.colors)
# choose simulation
runid='Arc12'
# choose depth
get3d = False
# variable ID
var_id= 'tr18'

########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/'
savepath    = '/work/ollie/loziel/figures/runoff/'
meshpath    = '/work/ollie/loziel/mesh/meshArc4.5_notrotated/'


# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /work/ollie/loziel/figures/runoff/ already existing


In [12]:
mesh = pf.load_mesh(meshpath, abg = [0., 0., 0.], usepickle=False,get3d=get3d)

Load 2d part of the grid in 36 second(s)


In [13]:
mesh


FESOM mesh:
path                  = /work/ollie/loziel/mesh/meshArc4.5_notrotated
alpha, beta, gamma    = 0.0, 0.0, 0.0
number of 2d nodes    = 753179
number of 2d elements = 1480268
number of 3d nodes    = 0

        

In [14]:
elem2=mesh.elem[mesh.no_cyclic_elem,:]

In [ ]:
# choose date
first_year = 1958
last_year  = 2019
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)

In [ ]:
for year in years:
    for mo in months:
        # loop over years
        print year
        ncfile = resultpath+'runoff_JRA55_monthly_'+str(year)+'.nc'
        print ncfile
        f      = Dataset(ncfile, 'r')
        runoff    = f.variables['runoff'][mo,0:mesh.n2d]
        ##################
        data2 = runoff
        print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
        print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
        print 'Mean of surface: ',np.mean(data2)
        print 'Max and min: ',np.max(data2),np.min(data2)
        
        
        contours = [0, 0.001, 0.0001]
        contours = np.arange(contours[0], contours[1]+contours[2], contours[2])
        #contours=np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
        fig = plt.figure(figsize=(10, 15), facecolor='w', edgecolor='k')
        #fig.suptitle('Density (kg m$^{-3}$)')
        fig.subplots_adjust(wspace=0.02,hspace=0.02)
        elem2=mesh.elem[mesh.no_cyclic_elem,:]

        d=data2[elem2].mean(axis=1)
        k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
        elem2=elem2[k,:]
        print 'ftriplot, number of dummy points:', len(d)-len(k)		
        map = Basemap(projection='nplaea',boundinglat=65,lon_0=0,resolution='l')
        x, y = map(mesh.x2, mesh.y2)
        #map.drawcoastlines()
        plabels=[False,False,False,False]
        mlabels=[False,False,False,False]    
        map.drawparallels(np.arange(-80.,81.,20.), labels=plabels)
        map.drawmeridians(np.arange(-180.,181.,20.),labels=mlabels) #[0,1,0,0]
        map.drawmapboundary(fill_color='0.9')
        map.fillcontinents(color='.5',lake_color='.7')
        eps=(contours.max()-contours.min())/100.
        data2[data2<=contours.min()]=contours.min()+eps
        data2[data2>=contours.max()]=contours.max()-eps
        im=plt.tricontourf(x, y, elem2, data2, levels=contours, cmap=cmap, norm=PiecewiseNorm(contours))

        cbar_ax = fig.add_axes([0.14, 0.2, 0.75, 0.02])
        cb = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
        cb.set_label(label='runoff', size='large')
        cb.ax.tick_params(labelsize='large')
        #plt.show() 
        if mo < 9:
            plt.savefig(savepath+'runoff_'+str(year)+'0'+str(mo+1)+'.png', dpi = 150, bbox_inches='tight')
        else:
            plt.savefig(savepath+'runoff_'+str(year)+str(mo+1)+'.png', dpi = 150, bbox_inches='tight')
        plt.close()

1958
/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/runoff_JRA55_monthly_1958.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  4.97575019887e-06
Max and min:  0.00653110931192 0.0
ftriplot, number of dummy points: 0


/global/AWIsoft/intel/2018/intelpython2/lib/python2.7/site-packages/mpl_toolkits/basemap/__init__.py:1767: MatplotlibDeprecationWarning: The get_axis_bgcolor function was deprecated in version 2.0. Use get_facecolor instead.
  axisbgc = ax.get_axis_bgcolor()


1958
/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/runoff_JRA55_monthly_1958.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  3.68312151721e-06
Max and min:  0.00954455565647 0.0
ftriplot, number of dummy points: 0
1958
/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/runoff_JRA55_monthly_1958.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  2.89690417936e-06
Max and min:  0.0134464667965 0.0
ftriplot, number of dummy points: 0
1958
/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/runoff_JRA55_monthly_1958.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  4.27383474157e-06
Max and min:  0.0175601485799 0.0
ftriplot, number of dummy points: 0
1958
/work/ollie/loziel/mesh/meshArc4.5_notrotated/runoff/runoff_JRA55_monthly_1958.nc
Number of nans in tracer:  0
Number of inf in tracer:  0
Mean of surface:  1.15654214073e-05
Max and min:  0.0209649506715 0.0
ftriplot, number of dummy points: 0